In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1= pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
df2= pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
df=pd.concat([df1, df2], axis=1)
df = df.rename(columns={"98.6":"O2Sat"})

In [ ]:
df.isna().count()

In [ ]:
df.describe()

In [ ]:
cols= df.columns
num_cols = df._get_numeric_data().columns
char_cols=list(set(cols) - set(num_cols))
char_cols

In [ ]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
def impute(df):
    df[num_cols]=si.fit_transform(df[num_cols])
    return df
def alter(df):
    for i in range(len(df.iloc[:,-1])):
        if df.iloc[:,-1][i]>0.5:
            df.iloc[:,-1][i]=1
        else:
            df.iloc[:,-1][i]=0
    return df

In [ ]:
df=impute(df)
df=alter(df)

In [ ]:
df

In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,stratify=df[['output']],shuffle=True)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
clf=RandomForestClassifier()
clf.fit(X_train,y_train)
y_pred= clf.predict(X_test)
print(f1_score(y_pred,y_test))

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score
clf=SVC()
clf.fit(X_train,y_train)
y_pred= clf.predict(X_test)
print(f1_score(y_pred,y_test))

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
clf=XGBClassifier()
clf.fit(X_train,y_train)
y_pred= clf.predict(X_test)
print(f1_score(y_pred,y_test))

In [ ]:
!pip install openpyxl
dfk = pd.DataFrame(X_test)
dfk['Pred']= y_pred
data = dfk.to_excel("outputXGBoost.xlsx",index=False,header=True)